In [ ]:
import pandas as pd
from pycaret.regression import *
import os

df = pd.read_csv(r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\smoothed_data_with_interactions.csv")

# 2) แปลง timestamp เป็น datetime และเรียงตามเวลา
df['date'] = pd.to_datetime(df['day'])
df['date_ordinal'] = df['date'].map(pd.Timestamp.toordinal) 
df.sort_values(by='date_ordinal', inplace=True)

# 3) สร้างคอลัมน์วันแบบตัวเลข (ordinal) และวัน/เดือน/ปี จาก date
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# ------------ NEW: สร้าง Lag Feature ----------------
# lag1 = ค่าเมื่อวันก่อนหน้า, lag2 = 2 วันก่อนหน้า
df['pm2_5_lag1'] = df['pm_2_5'].shift(1)
df['pm2_5_lag2'] = df['pm_2_5'].shift(2)

df['humidity_lag1'] = df['humidity'].shift(1)
df['humidity_lag2'] = df['humidity'].shift(2)

df['temperature_lag1'] = df['temperature'].shift(1)
df['temperature_lag2'] = df['temperature'].shift(2)


df.dropna(inplace=True)

data = df[[
    'day', 'month', 'year', 'date_ordinal',
    'humidity', 'temperature',
    'pm2_5_lag1', 'pm2_5_lag2',
    'humidity_lag1', 'humidity_lag2',
    'temperature_lag1', 'temperature_lag2','temperature humidity',
    'pm_2_5'  # target
]].copy()

reg_setup = setup(
    data=data,
    target='pm_2_5',
    fold_strategy='timeseries',
    fold=3,
    session_id=123,
    numeric_features=[
    'day', 'month', 'year', 'date_ordinal',
    'humidity', 'temperature',
    'pm2_5_lag1', 'pm2_5_lag2',
    'humidity_lag1', 'humidity_lag2',
    'temperature_lag1', 'temperature_lag2','temperature humidity'
    ],
    data_split_shuffle=False,
    fold_shuffle=False,
    use_gpu=True
)

# 6) เปรียบเทียบโมเดล
best_model = compare_models()

# 7) ปรับจูนโมเดลที่ดีที่สุด
tuned_model = tune_model(
    best_model,
    optimize='MAE',    
    n_iter=10000,     
    choose_better=True
)

# 8) Finalize โมเดล
final_model = finalize_model(tuned_model)

# 9) ประเมินบนข้อมูลเดิม (หรือ unseen data ถ้ามี)
pred_results = predict_model(final_model)
print(pred_results.head(7))


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

,Description,Value
0,Session id,123
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(27448, 14)"
4,Transformed data shape,"(27448, 14)"
5,Transformed train set shape,"(19213, 14)"
6,Transformed test set shape,"(8235, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,2.2067,10.5476,3.2443,0.8250,0.2024,0.1596,0.0333
ridge,Ridge Regression,2.2067,10.5477,3.2443,0.8250,0.2024,0.1596,0.0300
br,Bayesian Ridge,2.2068,10.5555,3.2455,0.8248,0.2024,0.1597,0.0300
lightgbm,Light Gradient Boosting Machine,2.2529,10.7972,3.2781,0.8223,0.2035,0.1625,1.4033
en,Elastic Net,2.2437,10.7534,3.2761,0.8219,0.2045,0.1633,0.0433
gbr,Gradient Boosting Regressor,2.2317,10.7660,3.2766,0.8218,0.2035,0.1619,4.1600
lasso,Lasso Regression,2.2456,10.7929,3.2820,0.8213,0.2051,0.1638,0.0433
llar,Lasso Least Angle Regression,2.2458,10.7931,3.2820,0.8213,0.2051,0.1638,0.0367
lar,Least Angle Regression,2.3188,11.0890,3.3282,0.8148,0.2183,0.1693,0.0267
rf,Random Forest Regressor,2.3174,11.3710,3.3667,0.8122,0.2082,0.1666,2.1467


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.3882,11.7524,3.4282,0.8475,0.2226,0.1815
1,2.2075,10.5115,3.2421,0.7755,0.1954,0.1486
2,2.0245,9.3789,3.0625,0.8519,0.1893,0.1487
Mean,2.2067,10.5476,3.2443,0.8250,0.2024,0.1596
Std,0.1485,0.9693,0.1493,0.0350,0.0145,0.0155


Fitting 3 folds for each of 2 candidates, totalling 6 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,1.8025,6.7928,2.6063,0.7416,0.2316,0.2068


      day  month  year  date_ordinal   humidity  temperature  pm2_5_lag1  \
2737    1      1  1970        719163  83.601341    27.606781    9.983665   
2736    1      1  1970        719163  84.726440    27.354673    8.639278   
2759    1      1  1970        719163  79.569267    28.779432    7.745468   
2735    1      1  1970        719163  85.609428    26.681051    8.612268   
2760    1      1  1970        719163  81.980881    28.563004    6.961753   
2762    1      1  1970        719163  82.136131    27.700129    7.855756   
2785    1      1  1970        719163  87.635567    26.677034    7.987162   

      pm2_5_lag2  humidity_lag1  humidity_lag2  temperature_lag1  \
2737    9.854004      82.723618      83.773270         28.156425   
2736    9.983665      83.601341      82.723618         27.606781   
2759    8.639278      84.726440      83.601341         27.354673   
2735    7.745468      79.569267      84.726440         28.779432   
2760    8.612268      85.609428      79.569267     

In [5]:
from pycaret.regression import save_model

save_model(final_model, 'final_pm2_5_model')


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['day', 'month', 'year',
                                              'date_ordinal', 'humidity',
                                              'temperature', 'pm2_5_lag1',
                                              'pm2_5_lag2', 'humidity_lag1',
                                              'humidity_lag2',
                                              'temperature_lag1',
                                              'temperature_lag2',
                                              'temperature humidity'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames()))

In [59]:
loaded_model = load_model('final_pm2_5_model')


Transformation Pipeline and Model Successfully Loaded


In [ ]:
from pycaret.regression import load_model, predict_model
import pandas as pd

# โหลดโมเดลที่ถูกฝึกไว้
model = load_model('final_pm2_5_model')

# โหลดข้อมูลใหม่
new_data = pd.read_csv(r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\Data_type\file_1d\unknow6_1d_processed.csv")

# ตรวจสอบว่าข้อมูลที่โหลดมามีฟีเจอร์อะไรบ้าง
print("Columns in new_data:", new_data.columns)

# ตรวจสอบว่าโมเดลต้องการฟีเจอร์อะไรบ้าง
print("Model features:", model.feature_names_in_)

# ค้นหาฟีเจอร์ที่ขาดหายไป
missing_features = [col for col in model.feature_names_in_ if col not in new_data.columns]
print("Missing features:", missing_features)

# เติมค่าฟีเจอร์ที่ขาดหายไป (ถ้าจำเป็น)
for missing_col in missing_features:
    new_data[missing_col] = 0  

# คัดเลือกเฉพาะฟีเจอร์ที่ตรงกับโมเดล
available_features = list(set(model.feature_names_in_) & set(new_data.columns))
filtered_data = new_data[available_features].copy()

# แสดงฟีเจอร์ที่ใช้สำหรับการพยากรณ์
print("Features used for prediction:", available_features)

# ทำการพยากรณ์ค่า PM2.5
predictions = predict_model(model, data=filtered_data)

# ตรวจสอบคอลัมน์ที่ได้จากการพยากรณ์
print("Columns in predictions:", predictions.columns)

# ตรวจสอบผลลัพธ์
print(predictions.head())
  
if 'prediction_label' in predictions.columns:
    print(predictions[['prediction_label']])  # ค่าที่พยากรณ์ได้
else:
    print("Error: 'prediction_label' column not found in predictions")


Transformation Pipeline and Model Successfully Loaded
Columns in new_data: Index(['day', 'month', 'year', 'date_ordinal', 'humidity', 'temperature',
       'pm2_5_lag1', 'pm2_5_lag2', 'humidity_lag1', 'humidity_lag2',
       'temperature_lag1', 'temperature_lag2', 'pm_2_5', 'temperature.1',
       'humidity.1', 'temperature humidity'],
      dtype='object')
Model features: ['day', 'month', 'year', 'date_ordinal', 'humidity', 'temperature', 'pm2_5_lag1', 'pm2_5_lag2', 'humidity_lag1', 'humidity_lag2', 'temperature_lag1', 'temperature_lag2', 'temperature humidity', 'pm_2_5']
Missing features: []
Features used for prediction: ['pm2_5_lag2', 'day', 'temperature_lag1', 'temperature', 'month', 'date_ordinal', 'temperature_lag2', 'pm_2_5', 'humidity', 'pm2_5_lag1', 'temperature humidity', 'humidity_lag1', 'year', 'humidity_lag2']


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,3.7842,29.0490,5.3897,0.7559,0.5231,14.7206


Columns in predictions: Index(['pm2_5_lag2', 'day', 'temperature_lag1', 'temperature', 'month',
       'date_ordinal', 'temperature_lag2', 'humidity', 'pm2_5_lag1',
       'temperature humidity', 'humidity_lag1', 'year', 'humidity_lag2',
       'pm_2_5', 'prediction_label'],
      dtype='object')
   pm2_5_lag2  day  temperature_lag1  temperature  month  date_ordinal  \
0   20.622437   14         27.396414    27.166878      2        737835   
1   16.708035   15         27.166878    27.031860      2        737836   
2   32.790688   16         27.031860    26.963314      2        737837   
3   33.745304   17         26.963314    26.270649      2        737838   
4   35.852329   18         26.270649    26.209059      2        737839   

   temperature_lag2   humidity  pm2_5_lag1  temperature humidity  \
0         27.426582  80.016808   16.708035           2173.806885   
1         27.396414  80.238754   32.790688           2169.002930   
2         27.166878  79.927254   33.745304           